In [2]:
import pandas as pd
from catboost import CatBoostClassifier
import gradio as gr
from gradio.components import File, Dataframe

# Определяем пути к моделям
SEX_MODEL_PATH = r'weights/29.09.050_sex_model.cbm'  
AGE_MODEL_PATH = r'weights/29.09.050_age_model.cbm'  

def load_models():
    sex_model = CatBoostClassifier()
    sex_model.load_model(SEX_MODEL_PATH)  # Загрузка модели пола
    age_model = CatBoostClassifier()
    age_model.load_model(AGE_MODEL_PATH)  # Загрузка модели возраста
    return sex_model, age_model

def create_features_and_predict(df):
    # Проверка входных данных
    if not all(col in df.columns for col in ['event_timestamp', 'region', 'ua_device_type',
                                              'ua_client_type', 'ua_os', 'ua_client_name', 
                                              'total_watchtime', 'rutube_video_id', 'viewer_uid']):
        raise ValueError("Input dataframe must contain specified columns.")

    # Создание нового столбца, который содержит количество просмотров для каждого пользователя
    df['view_count'] = df.groupby('viewer_uid')['viewer_uid'].transform('count')

    # Создать новый признак, который будет представлять собой суммарное время просмотра видео для каждого пользователя
    df['total_watchtime_per_user'] = df.groupby('viewer_uid')['total_watchtime'].transform('sum')

    # Выбор признаков для предсказания
    features = df[['region', 'ua_device_type', 'ua_client_type', 'ua_client_name',
                   'total_watchtime', 'rutube_video_id', 'viewer_uid', 'view_count',
                   'total_watchtime_per_user']].copy()

    # Загрузка моделей
    sex_model, age_model = load_models()

    # Предсказания пола
    sex_predictions = sex_model.predict(features)
    age_predictions = age_model.predict(features)

    # Добавление предсказанных значений в DataFrame
    df['sex'] = sex_predictions
    df['age_class'] = age_predictions

    # Выбор необходимых столбцов для сохранения
    submission = df[['viewer_uid', 'sex', 'age_class']].drop_duplicates()

    # Сохранение в файл submission.csv
    submission.to_csv('submission.csv', index=False)
    
    return submission

def load_data(file):
    # Загрузка данных
    df = pd.read_csv(file.name)
    submission = create_features_and_predict(df)

    # Возврат DataFrame для отображения
    return submission

# Определить интерфейс пользователя
def interface():
    inputs = File(label="Выберите файл c данными для загрузки", file_types=['.csv', '.xlsx'])
    outputs = Dataframe(label="Результаты предсказания", interactive=True)  # Добавлено interactive=True
    return gr.Interface(fn=load_data, inputs=inputs, outputs=outputs, title="Предсказание пола и возраста")

# Запустить интерфейс пользователя
if __name__ == '__main__':  
    app = interface()
    app.launch()


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
